In [ ]:
import torch
from dataset.get_datasets import get_dataset
import warnings

from dataset.scaffold import ogbg_with_smiles
import os 
import gzip
import pandas as pd

import sys
sys.path.append('./plotter/')
from plotter.plot import Plotter, convert_idx_list, Orig_Plotter

warnings.filterwarnings('ignore')

ImportError: cannot import name 'Plotter' from 'plotter.plot' (/home/allenwang/Projects/data_centric_transfer_scaffold/plotter/plot.py)

In [ ]:
labeled_dataset = get_dataset(args, './raw_data')
labeled_dataset_list = [data for data in labeled_dataset]
smile_path = os.path.join('./raw_data', '_'.join(args.dataset.split('-')), 'mapping/mol.csv.gz')
smiles_df = pd.read_csv(smile_path, compression='gzip', usecols=['smiles'])
smiles = smiles_df['smiles'].tolist() 
#test_idx = labeled_dataset.get_idx_split()

In [ ]:
new_labeled_dataset = ogbg_with_smiles(name = args.dataset,
                                   root = './raw_data',
                                   data_list = labeled_dataset_list, 
                                   smile_list = smiles)
label_split_idx = new_labeled_dataset.get_idx_split(split_type = 'scaffold')
label_split_idx_random = new_labeled_dataset.get_idx_split(split_type = 'random')

## TSNE

In [ ]:
split_list = convert_idx_list(label_split_idx)
split_list_random = convert_idx_list(label_split_idx_random)
# append ths split_list to smile datframe
smiles_df['split'] = split_list
#smiles_df['y'] = y_list
smiles_df['split_random'] = split_list_random

In [ ]:
smiles_df.head(5)

In [ ]:

# test= Plotter.from_smiles(smiles_df["smiles"], target=smiles_df["y"], idx = smiles_df["split"] , 
#                           target_type="C", sim_type='structural')

test1 = Orig_Plotter.from_smiles(smiles_df["smiles"], target=smiles_df["split"] ,target_type="C", sim_type='structural')
#test2 = Orig_Plotter.from_smiles(smiles_df["smiles"], target=smiles_df["y"] ,target_type="C", sim_type='structural')
test3 = Orig_Plotter.from_smiles(smiles_df["smiles"], target=smiles_df["split_random"] ,target_type="C", sim_type='structural')


In [ ]:
test1.tsne(random_state=0)
#test2.tsne(random_state=0)
test3.tsne(random_state=0)

In [ ]:
fig1 = test1.visualize_plot(kind = 'scatter', size = 11, title=f'{args.dataset} scaffold split t-sne')
fig3 = test3.visualize_plot(kind = 'scatter', size = 11, title=f'{args.dataset} random split t-sne')
#fig2 = test2.visualize_plot(kind = 'scatter', size = 11, title=f'{args.dataset} property t-sne')


In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
test1.interactive_plot(show_plot=True)

In [ ]:
test2.interactive_plot(show_plot=True)

## Scaffold Data

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)    

class Args:
    def __init__(self):
        self.dataset = 'ogbg-molbace'
        self.device = device
        self.batch_size = 128
        
args = Args()


labeled_dataset = get_dataset(args, './raw_data')
labeled_dataset_list = [data for data in labeled_dataset]
smile_path = os.path.join('./raw_data', '_'.join(args.dataset.split('-')), 'mapping/mol.csv.gz')
smiles_df = pd.read_csv(smile_path, compression='gzip', usecols=['smiles'])
smiles = smiles_df['smiles'].tolist() 


In [ ]:
from dataset.scaffold import get_scaffold_split_info, _generate_scaffold, generate_scaffolds_dict

label_scaffold_split_idx = new_labeled_dataset.get_idx_split(split_type = 'scaffold')

_,all_scaffolds_set = generate_scaffolds_dict(smiles)
print(f'There are {len(all_scaffolds_set)} scaffolds in total')

all_scaffolds_smiles = [tup[0] for tup in all_scaffolds_set]

In [ ]:
train_smiles = [smiles[i] for i in label_scaffold_split_idx['train']]
_, train_scaffold = generate_scaffolds_dict(train_smiles)
train_scaffold_num = dict([(tup[0], len(tup[1])) for tup in train_scaffold])
# sort the dict by alphabetic order of the scaffold smiles
#train_scaffold_num = dict(sorted(train_scaffold_num.items(), key=lambda item: item[0]))

valid_smiles = [smiles[i] for i in label_scaffold_split_idx['valid']]
_, valid_scaffold = generate_scaffolds_dict(valid_smiles)
valid_scaffold_num = dict([(tup[0], len(tup[1])) for tup in valid_scaffold])
#valid_scaffold_num = dict(sorted(valid_scaffold_num.items(), key=lambda item: item[0]))

test_smiles = [smiles[i] for i in label_scaffold_split_idx['test']]
_, test_scaffold = generate_scaffolds_dict(test_smiles)
test_scaffold_num = dict([(tup[0], len(tup[1])) for tup in test_scaffold])
#test_scaffold_num = dict(sorted(test_scaffold_num.items(), key=lambda item: item[0]))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(18, 6))

# Plot for train set
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
keys = list(train_scaffold_num.keys())
counts = list(train_scaffold_num.values())
indices = list(range(len(keys)))
plt.bar(indices, counts)
plt.xlabel('scaffold id')
plt.ylabel('count')
plt.ylim(0, 60)  # Set y limit
plt.title('scaffold distribution in train set')

# Plot for test set
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
keys = list(test_scaffold_num.keys())
counts = list(test_scaffold_num.values())
indices = list(range(len(keys)))
plt.bar(indices, counts)
plt.xlabel('scaffold id')
plt.ylim(0, 60)  # Set y limit
plt.title('scaffold distribution in test set')

# Plot for validation set
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
keys = list(valid_scaffold_num.keys())
counts = list(valid_scaffold_num.values())
indices = list(range(len(keys)))
plt.bar(indices, counts)
plt.xlabel('scaffold id')
plt.ylim(0, 60)  # Set y limit
plt.title('scaffold distribution in validation set')

plt.tight_layout()
plt.show()

In [ ]:
scaffold_split_dict = {}
scaffold_num_dict = {}
for scaffold_smile in all_scaffolds_smiles:
    if train_scaffold_num[scaffold_smile] > 0:
        scaffold_split_dict[scaffold_smile] = 'train'
        scaffold_num_dict[scaffold_smile] = train_scaffold_num[scaffold_smile]
    elif valid_scaffold_num[scaffold_smile] > 0:
        scaffold_split_dict[scaffold_smile] = 'valid'
        scaffold_num_dict[scaffold_smile] = valid_scaffold_num[scaffold_smile]
    else:
        scaffold_split_dict[scaffold_smile] = 'test'
        scaffold_num_dict[scaffold_smile] = test_scaffold_num[scaffold_smile]


In [ ]:
scaffold_split_df = pd.DataFrame.from_dict(scaffold_split_dict, orient='index')
scaffold_split_df.reset_index(inplace=True)

scaffold_split_df.columns = ['scaffold', 'split']

scaffold_split_df['num'] = scaffold_split_df['scaffold'].map(scaffold_num_dict)

print(scaffold_split_df['split'].value_counts())
scaffold_split_df.head()

In [ ]:
from plotter.plot_test import Plotter
plot = Orig_Plotter.from_smiles(scaffold_split_df["scaffold"], target=scaffold_split_df["split"] ,target_type="C", sim_type='structural')
plot.tsne(random_state=0)
plot.visualize_plot(kind = 'scatter', size = 11, title=f'{args.dataset} scaffold split t-sne')